# Preparación del entorno

In [ ]:
import os
import pandas as pd

In [ ]:
# Aquí se debe poner la carpeta de la computadora local
carpeta_local = "~/buckets/b1/"

# Establezco el Working Directory
os.chdir(os.path.expanduser(carpeta_local))

# Cargo el dataset
archivo_dataset = "./datasets/competencia_03 (base).csv.gz"
dataset = pd.read_csv(archivo_dataset)


# Prueba con SQL

In [ ]:
%%bash
pip install --upgrade duckdb
pip install --upgrade jupysql
pip install --upgrade duckdb-engine

In [ ]:
import duckdb
import pandas as pd

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:///:default:

# Construcción de Clase_ternaria

In [ ]:
%%sql
create or replace table competencia_03 as 
select 
    *
from read_csv_auto("./datasets/competencia_03.csv.gz")

In [ ]:
%sql select * from competencia_03 limit 5

In [ ]:
%%sql
select
    foto_mes
    , count(*) as cantidad -- cuenta cuantos casos hay en cada foto_mes
                           -- y lo guarda en un campo llamado cantidad
from competencia_03
group by foto_mes

# Descarga índice de inflación

## Armado del índice

In [ ]:
!pip install pandas requests

In [ ]:
import requests
from io import BytesIO

In [ ]:
# URL del archivo
url = "https://www.indec.gob.ar/ftp/cuadros/economia/sh_ipc_08_23.xls"

# Descargar el archivo y cargar los datos de la hoja correcta
response = requests.get(url)
xls = pd.ExcelFile(BytesIO(response.content))

# Cargar los datos desde la hoja de interés
inflacion = pd.read_excel(xls, sheet_name="Índices IPC Cobertura Nacional", skiprows=5, usecols="B:CC")

In [ ]:
inflacion

In [ ]:
# Transponer los datos y establecer la tercera fila como encabezado
inflacion = inflacion.transpose()


In [ ]:
inflacion = inflacion[3]

In [ ]:
inflacion

In [ ]:
# Pasar a data frame
import pandas as pd

inflacion = pd.DataFrame(inflacion)

In [ ]:
inflacion.index.name='foto_mes'

In [ ]:
inflacion

In [ ]:
# Renombrar la columna
inflacion.columns = ["indice_inflacion"]

# Eliminar las filas con valores nulos
inflacion.dropna(inplace=True)

# Eliminar guiones y los últimos dos dígitos de la columna "foto_mes"
inflacion.index = inflacion.index.astype(str).str.replace('-', '').str[:-2]

# Mostrar las primeras filas del DataFrame resultante
inflacion.head()

## Inflación A Sql

In [ ]:
# Guardar el DataFrame como un archivo CSV
inflacion.to_csv('inflacion.csv')

In [ ]:
%%sql
create or replace table inflacion as
select
    *
from read_csv_auto('inflacion.csv')

In [ ]:
%%sql
select *
from inflacion

Agrego inflación acumulada hasta 202107 para hacer el drifting

In [ ]:
%%sql
ALTER TABLE inflacion
ADD COLUMN indice_inflacion_acumulada FLOAT; -- Adjust the data type as needed

WITH InflacionAcumulada AS (
    SELECT
        foto_mes,
        indice_inflacion,
        (
            (SELECT indice_inflacion FROM inflacion WHERE foto_mes = '202107') / indice_inflacion
        ) AS inflacion_acumulada
    FROM
        inflacion
)
UPDATE inflacion
SET indice_inflacion_acumulada = (SELECT inflacion_acumulada FROM InflacionAcumulada WHERE inflacion.foto_mes = InflacionAcumulada.foto_mes);


In [ ]:
%%sql
select *
from inflacion

# Inner join competencia_03 e inflación

In [ ]:
%%sql
CREATE TABLE competencia_03_temp AS
SELECT
    *
FROM
    competencia_03 c2
INNER JOIN
    inflacion i
ON
    c2.foto_mes = i.foto_mes;

In [ ]:
%%sql
-- Elimina la columna 'indice_inflacion' de la tabla temporal
ALTER TABLE competencia_03_temp
DROP COLUMN indice_inflacion;

-- Borra la tabla original 'competencia_03'
DROP TABLE competencia_03;

-- Renombra la tabla temporal a 'competencia_03'
ALTER TABLE competencia_03_temp
RENAME TO competencia_03;

In [ ]:
%%sql
SELECT
    *
FROM
    competencia_03 c2

# Solución del Data Drifting a variables monetarias

Primero, ajustos todas las variables monetarias por la inflación acumulada a 202107

In [ ]:
%%sql
ALTER TABLE competencia_03
ADD COLUMN Mrentabilidad_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mrentabilidad_annual_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mcomisiones_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mactivos_margen_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mpasivos_margen_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mcuenta_corriente_adicional_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mcuenta_corriente_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mcaja_ahorro_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mcaja_ahorro_adicional_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mcaja_ahorro_dolares_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mcuentas_saldo_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mautoservicio_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mtarjeta_visa_consumo_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mtarjeta_master_consumo_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mprestamos_personales_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mprestamos_prendarios_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mprestamos_hipotecarios_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mplazo_fijo_dolares_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mplazo_fijo_pesos_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN minversion1_pesos_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN minversion1_dolares_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN minversion2_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mpayroll_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mpayroll2_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mcuenta_debitos_automaticos_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mttarjeta_master_debitos_automaticos_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mpagodeservicios_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mpagomiscuentas_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mcajeros_propios_descuentos_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mtarjeta_visa_descuentos_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mtarjeta_master_descuentos_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mcomisiones_mantenimiento_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mcomisiones_otras_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mforex_buy_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mforex_sell_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mtransferencias_recibidas_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mtransferencias_emitidas_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mextraccion_autoservicio_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mcheques_depositados_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mcheques_emitidos_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mcheques_depositados_rechazados_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mcheques_emitidos_rechazados_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN matm_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN matm_other_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN Master_mfinanciacion_limite_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN Master_msaldototal_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN Master_msaldopesos_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN Master_msaldodolares_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN Master_mconsumospesos_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN Master_mconsumosdolares_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN Master_mlimitecompra_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN Master_madelantopesos_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN Master_madelantodolares_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN Master_mpagado_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN Master_mpagospesos_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN Master_mpagosdolares_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN Master_mconsumototal_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN Master_mpagominimo_ajustado DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN Visa_mfinanciacion_limite_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN Visa_msaldototal_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN Visa_msaldopesos_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN Visa_msaldodolares_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN Visa_mconsumospesos_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN Visa_mconsumosdolares_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN Visa_mlimitecompra_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN Visa_madelantopesos_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN Visa_madelantodolares_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN Visa_mpagado_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN Visa_mpagospesos_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN Visa_mpagosdolares_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN Visa_mconsumototal_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN Visa_mpagominimo_ajustado DECIMAL(18, 2);

In [ ]:
%%sql
UPDATE competencia_03
SET
    Mrentabilidad_ajustada = Mrentabilidad * indice_inflacion_acumulada,
    mrentabilidad_annual_ajustada = mrentabilidad_annual * indice_inflacion_acumulada,
    mcomisiones_ajustada = mcomisiones * indice_inflacion_acumulada,
    mactivos_margen_ajustada = mactivos_margen * indice_inflacion_acumulada,
    mpasivos_margen_ajustada = mpasivos_margen * indice_inflacion_acumulada,
    mcuenta_corriente_adicional_ajustada = mcuenta_corriente_adicional * indice_inflacion_acumulada,
    mcuenta_corriente_ajustada = mcuenta_corriente * indice_inflacion_acumulada,
    mcaja_ahorro_ajustada = mcaja_ahorro * indice_inflacion_acumulada,
    mcaja_ahorro_adicional_ajustada = mcaja_ahorro_adicional * indice_inflacion_acumulada,
    mcaja_ahorro_dolares_ajustada = mcaja_ahorro_dolares * indice_inflacion_acumulada,
    mcuentas_saldo_ajustada = mcuentas_saldo * indice_inflacion_acumulada,
    mautoservicio_ajustada = mautoservicio * indice_inflacion_acumulada,
    mtarjeta_visa_consumo_ajustada = mtarjeta_visa_consumo * indice_inflacion_acumulada,
    mtarjeta_master_consumo_ajustada = mtarjeta_master_consumo * indice_inflacion_acumulada,
    mprestamos_personales_ajustada = mprestamos_personales * indice_inflacion_acumulada,
    mprestamos_prendarios_ajustada = mprestamos_prendarios * indice_inflacion_acumulada,
    mprestamos_hipotecarios_ajustada = mprestamos_hipotecarios * indice_inflacion_acumulada,
    mplazo_fijo_dolares_ajustada = mplazo_fijo_dolares * indice_inflacion_acumulada,
    mplazo_fijo_pesos_ajustada = mplazo_fijo_pesos * indice_inflacion_acumulada,
    minversion1_pesos_ajustada = minversion1_pesos * indice_inflacion_acumulada,
    minversion1_dolares_ajustada = minversion1_dolares * indice_inflacion_acumulada,
    minversion2_ajustada = minversion2 * indice_inflacion_acumulada,
    mpayroll_ajustada = mpayroll * indice_inflacion_acumulada,
    mpayroll2_ajustada = mpayroll2 * indice_inflacion_acumulada,
    mcuenta_debitos_automaticos_ajustada = mcuenta_debitos_automaticos * indice_inflacion_acumulada,
    mttarjeta_master_debitos_automaticos_ajustada = mttarjeta_master_debitos_automaticos * indice_inflacion_acumulada,
    mpagodeservicios_ajustada = mpagodeservicios * indice_inflacion_acumulada,
    mpagomiscuentas_ajustada = mpagomiscuentas * indice_inflacion_acumulada,
    mcajeros_propios_descuentos_ajustada = mcajeros_propios_descuentos * indice_inflacion_acumulada,
    mtarjeta_visa_descuentos_ajustada = mtarjeta_visa_descuentos * indice_inflacion_acumulada,
    mtarjeta_master_descuentos_ajustada = mtarjeta_master_descuentos * indice_inflacion_acumulada,
    mcomisiones_mantenimiento_ajustada = mcomisiones_mantenimiento * indice_inflacion_acumulada,
    mcomisiones_otras_ajustada = mcomisiones_otras * indice_inflacion_acumulada,
    mforex_buy_ajustada = mforex_buy * indice_inflacion_acumulada,
    mforex_sell_ajustada = mforex_sell * indice_inflacion_acumulada,
    mtransferencias_recibidas_ajustada = mtransferencias_recibidas * indice_inflacion_acumulada,
    mtransferencias_emitidas_ajustada = mtransferencias_emitidas * indice_inflacion_acumulada,
    mextraccion_autoservicio_ajustada = mextraccion_autoservicio * indice_inflacion_acumulada,
    mcheques_depositados_ajustada = mcheques_depositados * indice_inflacion_acumulada,
    mcheques_emitidos_ajustada = mcheques_emitidos * indice_inflacion_acumulada,
    mcheques_depositados_rechazados_ajustada = mcheques_depositados_rechazados * indice_inflacion_acumulada,
    mcheques_emitidos_rechazados_ajustada = mcheques_emitidos_rechazados * indice_inflacion_acumulada,
    matm_ajustada = matm * indice_inflacion_acumulada,
    matm_other_ajustada = matm_other * indice_inflacion_acumulada,
    Master_mfinanciacion_limite_ajustada = Master_mfinanciacion_limite * indice_inflacion_acumulada,
    Master_msaldototal_ajustada = Master_msaldototal * indice_inflacion_acumulada,
    Master_msaldopesos_ajustada = Master_msaldopesos * indice_inflacion_acumulada,
    Master_msaldodolares_ajustada = Master_msaldodolares * indice_inflacion_acumulada,
    Master_mconsumospesos_ajustada = Master_mconsumospesos * indice_inflacion_acumulada,
    Master_mconsumosdolares_ajustada = Master_mconsumosdolares * indice_inflacion_acumulada,
    Master_mlimitecompra_ajustada = Master_mlimitecompra * indice_inflacion_acumulada,
    Master_madelantopesos_ajustada = Master_madelantopesos * indice_inflacion_acumulada,
    Master_madelantodolares_ajustada = Master_madelantodolares * indice_inflacion_acumulada,
    Master_mpagado_ajustada = Master_mpagado * indice_inflacion_acumulada,
    Master_mpagospesos_ajustada = Master_mpagospesos * indice_inflacion_acumulada,
    Master_mpagosdolares_ajustada = Master_mpagosdolares * indice_inflacion_acumulada,
    Master_mconsumototal_ajustada = Master_mconsumototal * indice_inflacion_acumulada,
    Master_mpagominimo_ajustado = Master_mpagominimo * indice_inflacion_acumulada,
    Visa_mfinanciacion_limite_ajustada = Visa_mfinanciacion_limite * indice_inflacion_acumulada,
    Visa_msaldototal_ajustada = Visa_msaldototal * indice_inflacion_acumulada,
    Visa_msaldopesos_ajustada = Visa_msaldopesos * indice_inflacion_acumulada,
    Visa_msaldodolares_ajustada = Visa_msaldodolares * indice_inflacion_acumulada,
    Visa_mconsumospesos_ajustada = Visa_mconsumospesos * indice_inflacion_acumulada,
    Visa_mconsumosdolares_ajustada = Visa_mconsumosdolares * indice_inflacion_acumulada,
    Visa_mlimitecompra_ajustada = Visa_mlimitecompra * indice_inflacion_acumulada,
    Visa_madelantopesos_ajustada = Visa_madelantopesos * indice_inflacion_acumulada,
    Visa_madelantodolares_ajustada = Visa_madelantodolares * indice_inflacion_acumulada,
    Visa_mpagado_ajustada = Visa_mpagado * indice_inflacion_acumulada,
    Visa_mpagospesos_ajustada = Visa_mpagospesos * indice_inflacion_acumulada,
    Visa_mpagosdolares_ajustada = Visa_mpagosdolares * indice_inflacion_acumulada,
    Visa_mconsumototal_ajustada = Visa_mconsumototal * indice_inflacion_acumulada,
    Visa_mpagominimo_ajustado = Visa_mpagominimo * indice_inflacion_acumulada;


Después,ya elimino la variable "inflación_acumulada" ya que no es de más utilidad, y las variables monetarias nominales

In [ ]:
%%sql
ALTER TABLE competencia_03
DROP COLUMN indice_inflacion_acumulada;

In [ ]:
%%sql
ALTER TABLE competencia_03
DROP COLUMN Mrentabilidad;

ALTER TABLE competencia_03
DROP COLUMN mrentabilidad_annual;

ALTER TABLE competencia_03
DROP COLUMN mcomisiones;

ALTER TABLE competencia_03
DROP COLUMN mactivos_margen;

ALTER TABLE competencia_03
DROP COLUMN mpasivos_margen;

ALTER TABLE competencia_03
DROP COLUMN mcuenta_corriente_adicional;

ALTER TABLE competencia_03
DROP COLUMN mcuenta_corriente;

ALTER TABLE competencia_03
DROP COLUMN mcaja_ahorro;

ALTER TABLE competencia_03
DROP COLUMN mcaja_ahorro_adicional;

ALTER TABLE competencia_03
DROP COLUMN mcaja_ahorro_dolares;

ALTER TABLE competencia_03
DROP COLUMN mcuentas_saldo;

ALTER TABLE competencia_03
DROP COLUMN mautoservicio;

ALTER TABLE competencia_03
DROP COLUMN mtarjeta_visa_consumo;

ALTER TABLE competencia_03
DROP COLUMN mtarjeta_master_consumo;

ALTER TABLE competencia_03
DROP COLUMN mprestamos_personales;

ALTER TABLE competencia_03
DROP COLUMN mprestamos_prendarios;

ALTER TABLE competencia_03
DROP COLUMN mprestamos_hipotecarios;

ALTER TABLE competencia_03
DROP COLUMN mplazo_fijo_dolares;

ALTER TABLE competencia_03
DROP COLUMN mplazo_fijo_pesos;

ALTER TABLE competencia_03
DROP COLUMN minversion1_pesos;

ALTER TABLE competencia_03
DROP COLUMN minversion1_dolares;

ALTER TABLE competencia_03
DROP COLUMN minversion2;

ALTER TABLE competencia_03
DROP COLUMN mpayroll;

ALTER TABLE competencia_03
DROP COLUMN mpayroll2;

ALTER TABLE competencia_03
DROP COLUMN mcuenta_debitos_automaticos;

--ALTER TABLE competencia_03
--DROP COLUMN mtarjeta_visa_debitos_automaticos;

ALTER TABLE competencia_03
DROP COLUMN mttarjeta_master_debitos_automaticos;

ALTER TABLE competencia_03
DROP COLUMN mpagodeservicios;

ALTER TABLE competencia_03
DROP COLUMN mpagomiscuentas;

ALTER TABLE competencia_03
DROP COLUMN mcajeros_propios_descuentos;

ALTER TABLE competencia_03
DROP COLUMN mtarjeta_visa_descuentos;

ALTER TABLE competencia_03
DROP COLUMN mtarjeta_master_descuentos;

ALTER TABLE competencia_03
DROP COLUMN mcomisiones_mantenimiento;

ALTER TABLE competencia_03
DROP COLUMN mcomisiones_otras;

ALTER TABLE competencia_03
DROP COLUMN mforex_buy;

ALTER TABLE competencia_03
DROP COLUMN mforex_sell;

ALTER TABLE competencia_03
DROP COLUMN mtransferencias_recibidas;

ALTER TABLE competencia_03
DROP COLUMN mtransferencias_emitidas;

ALTER TABLE competencia_03
DROP COLUMN mextraccion_autoservicio;

ALTER TABLE competencia_03
DROP COLUMN mcheques_depositados;

ALTER TABLE competencia_03
DROP COLUMN mcheques_emitidos;

ALTER TABLE competencia_03
DROP COLUMN mcheques_depositados_rechazados;

ALTER TABLE competencia_03
DROP COLUMN mcheques_emitidos_rechazados;

ALTER TABLE competencia_03
DROP COLUMN matm;

ALTER TABLE competencia_03
DROP COLUMN Master_mfinanciacion_limite;

ALTER TABLE competencia_03
DROP COLUMN Master_msaldototal;

ALTER TABLE competencia_03
DROP COLUMN Master_msaldopesos;

ALTER TABLE competencia_03
DROP COLUMN Master_msaldodolares;

ALTER TABLE competencia_03
DROP COLUMN Master_mconsumospesos;

ALTER TABLE competencia_03
DROP COLUMN Master_mconsumosdolares;

ALTER TABLE competencia_03
DROP COLUMN Master_mlimitecompra;

ALTER TABLE competencia_03
DROP COLUMN Master_madelantopesos;

ALTER TABLE competencia_03
DROP COLUMN Master_madelantodolares;

ALTER TABLE competencia_03
DROP COLUMN Master_mpagado;

ALTER TABLE competencia_03
DROP COLUMN Master_mpagospesos;

ALTER TABLE competencia_03
DROP COLUMN Master_mpagosdolares;

ALTER TABLE competencia_03
DROP COLUMN Master_mconsumototal;

ALTER TABLE competencia_03
DROP COLUMN Master_mpagominimo;

ALTER TABLE competencia_03
DROP COLUMN Visa_mfinanciacion_limite;

ALTER TABLE competencia_03
DROP COLUMN Visa_msaldototal;

ALTER TABLE competencia_03
DROP COLUMN Visa_msaldopesos;

ALTER TABLE competencia_03
DROP COLUMN Visa_msaldodolares;

ALTER TABLE competencia_03
DROP COLUMN Visa_mconsumospesos;

ALTER TABLE competencia_03
DROP COLUMN Visa_mconsumosdolares;

ALTER TABLE competencia_03
DROP COLUMN Visa_mlimitecompra;

ALTER TABLE competencia_03
DROP COLUMN Visa_madelantopesos;

ALTER TABLE competencia_03
DROP COLUMN Visa_madelantodolares;

ALTER TABLE competencia_03
DROP COLUMN Visa_mpagado;

ALTER TABLE competencia_03
DROP COLUMN Visa_mpagospesos;

ALTER TABLE competencia_03
DROP COLUMN Visa_mpagosdolares;

ALTER TABLE competencia_03
DROP COLUMN Visa_mconsumototal;

ALTER TABLE competencia_03
DROP COLUMN Visa_mpagominimo;


# Guardar el csv con las variables transformadas

In [ ]:
%%sql
copy competencia_03 to '/datasets/competencia_03_ajust_x_infla.csv.gz' (FORMAT CSV, HEADER)